# oneMKL GEMM Exercise (Using SYCL C++ Unified Shared Memory)

The following example shows a simple matrix multiplication program using __SYCL__ with the __unified shared memory__ style of memory management. Follow along with the instructions of the lab to build and run the program. 

In [9]:
%%writefile lab/dpcpp_gemm_usm.cpp
//==============================================================
// Copyright © 2023 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <iostream>
#include <vector>
#include <sycl/sycl.hpp>          //# sycl namespace
#include "oneapi/mkl/blas.hpp"  //# oneMKL DPC++ interface for BLAS functions

// # The following project performs matrix multiplication using oneMKL / DPC++ with Unified Shared Memory (USM)
// # We will execute the simple operation A * B = C
// # The matrix B is set equal to the identity matrix such that A * B = A * I
// # After performing the computation, we will verify A * I = C -> A = C

using namespace sycl;
namespace mkl = oneapi::mkl;  //# shorten mkl namespace

int main() {

    //# dimensions
    int m = 3, n = 3, k = 3;
    
    //# leading dimensions
    int ldA = 3, ldB = 3, ldC = 3;
    
    //# scalar multipliers
    float alpha = 1.0, beta = 1.0;
    
    //# transpose status of matrices
    mkl::transpose transA = mkl::transpose::nontrans;
    mkl::transpose transB = mkl::transpose::nontrans;

    //### Step 1 - Create a queue with default selector.
    queue q;
    device my_device = q.get_device();
    std::cout << "Device: " << my_device.get_info<info::device::name>() << "\n";

    //### Step 2 - Create a sycl event and allocate USM
    //# The later execution of the gemm operation is tied to this event
    //# The gemm operation will also make use of a vector of sycl events we can call 'gemm_dependencies'
    
    sycl::event gemm_done;
    std::vector<sycl::event> gemm_dependencies;
    
    //# Here, we allocate USM pointers for each matrix, using the special 'malloc_shared' function
    //# Make sure to template the function with the correct precision, and pass in our queue to the function call
    
    float *A_usm = sycl::malloc_shared<float>(m * k, q);
    float *B_usm = sycl::malloc_shared<float>(k * n, q);
    float *C_usm = sycl::malloc_shared<float>(m * n, q);

    //# define matrix A as the 3x3 matrix
    //# {{ 1, 2, 3}, {4, 5, 6}, {7, 8, 9}}
    
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < k; j++) {
            A_usm[i*m+j] = (float)(i*m+j) + 1.0;
        }
    }
    
    //# define matrix B as the identity matrix
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < n; j++) {
            if (i == j) B_usm[i*k+j] = 1.0;
            else B_usm[i*k+j] = 0.0;
        }
    }
    
    //# initialize C as a 0 matrix
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            C_usm[i*m+j] = 0.0;
        }
    }

    //### Step 3 - Execute gemm operation.
    //# Here, we fill in the familiar parameters for the gemm operation.
    //# However, we must also pass in the queue as the first parameter.
    //# We must also pass in our list of dependencies as the final parameter.
    //# We are also passing in our USM pointers as opposed to a buffer or raw data pointer.
    
    gemm_done = mkl::blas::gemm(q, transA, transB, m, n, k, alpha, A_usm, ldA, B_usm, ldB, beta, C_usm, ldC, gemm_dependencies);

    //# We must now wait for the given event to finish before accessing any data involved in the operation
    //# Otherwise, we may access data before the operation has completed, or before it has been returned to the host
    gemm_done.wait();

    int status = 0;

    //# verify C matrix using USM data
    std::cout << "\n";
    std::cout << "C = \n";
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            if (A_usm[i*m+j] != C_usm[i*m+j]) status = 1;
            std::cout << C_usm[i*m+j] << " ";
        }
        std::cout << "\n";
    }
    std::cout << "\n";

    //# free usm pointers
    sycl::free(A_usm, q);
    sycl::free(B_usm, q);
    sycl::free(C_usm, q);

    status == 0 ? std::cout << "Verified: A = C\n" : std::cout << "Failed: A != C\n";
    return status;
}

Writing lab/dpcpp_gemm_usm.cpp


### Build and Run
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 run_gemm_usm.sh;if [ -x "$(command -v qsub)" ]; then ./q run_gemm_usm.sh; else ./run_gemm_usm.sh; fi

The build instructions for this sample can be found in the ```run_gemm_usm.sh``` script. Consider using the [Link Line Advisor](https://software.intel.com/content/www/us/en/develop/articles/intel-mkl-link-line-advisor.html) to help you create compile and link lines for your oneMKL projects.

<html><body><span style="color:green"><h1>Survey</h1></span></body></html>

[We would appreciate any feedback you’d care to give, so that we can improve the overall training quality and experience. Thanks! ](https://intel.az1.qualtrics.com/jfe/form/SV_3JyBEAtwk5YIg85)

<html><body><span style="color:Red"><h1>Reset Notebook</h1></span></body></html>

##### Should you be experiencing any issues with your notebook or just want to start fresh run the below cell.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/Intel_oneAPI_MKL_Training/00_GEMM/ ~/Intel_oneAPI_MKL_Training/00_GEMM/
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])